# Occupational Coding Predict

In [ ]:
import pandas as pd # Read data and output

import jieba # Word segmentation
import jieba.analyse as analyse

import numpy as np # Data processing

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, MaxPool1D, Conv1D, Convolution1D
from keras.layers import Embedding
from keras import regularizers
from keras.layers import BatchNormalization
from keras.models import load_model

from gensim.models import KeyedVectors # Import pre-trained word vectos

import matplotlib.pyplot as plt # Draw graph

## Set Up
Set up for training the model<br>
Variables needed to be set: word2vec_path, stopwords_path, vec_size

In [ ]:
# Set the file paths needed and dimension of the pre-trained word vectors

stopwords_path = 'stopword.txt' # Stop-words file
word2vec_path = 'tencent-ailab-embedding-zh-d200-v0.2.0-s.txt' # Pre-trained word vectors file

vec_size = 200 # Deimension of the pre-trained word vectors

In [ ]:
# Import pre-trained word vectors

wv_from_text = KeyedVectors.load_word2vec_format(word2vec_path, binary=False)

In [ ]:
# Declare functions used

# Create list of stop-words
def stopwordslist(filepath):
    stopwords = [line.strip() for line in open(filepath, 'r', encoding='utf-8').readlines()]
    return stopwords
 
 
# Word segmentation
def seg_sentence(sentence):
    sentence_seged = jieba.lcut(sentence.strip())
    stopwords = stopwordslist(stopwords_path)
    outstr = []
    for word in sentence_seged:
        if word not in stopwords: # Delete the stop-words
            if word != '\t' and word != ' ': # Delete spaces
                outstr.append(word)
    return outstr

# Change the words segemented into word vectors
def transform_to_matrix(x, padding_size=10, vec_size=vec_size):
    res = []
    for sen in x:
        matrix = []
        for i in range(padding_size):
            try:
                matrix.append(wv_from_text[sen[i]].tolist())
            except:
                matrix.append([0] * vec_size)
        res.append(matrix)
    return res

## Coding Prediction
Variables needed to be set: predict_path, output_path, predict_name, model_path

In [ ]:
# Set file paths

predict_path = 'zy_predict.xlsx' # Data to be predicted
output_path = 'output_zy.xlsx' # Output the prediction result
model_path = 'model_zy.h5' # Path of the model

In [ ]:
# Import the model

model = load_model(model_path)

In [ ]:
# Import data to be predicted
predict_pd = pd.read_excel(predict_path, index_col=0)

# Set column name of the data to be predicted
predict_name = 'QG303'

# Data processing
x = predict_pd[predict_name]
x = np.array(x)
temp = []
for i in x:
    line_seg = seg_sentence(i)
    temp.append(line_seg)
x = temp

x = transform_to_matrix(x)

In [ ]:
# Call predict function to predict coding of the data
# Convert possibilities into exact codes
# Export prediction result to excel file

predict = model.predict(x)
predict_1 = predict.argmax(axis=-1)
predict_1 = np.array(predict_1)
te = pd.DataFrame(predict_1)
te.to_excel(output_path, sheet_name='Sheet1')